In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import numpy as np
from scipy.stats import entropy
from sklearn.metrics import r2_score
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
from time_model_utils import *

In [3]:
class LSTMModel1(nn.Module):
    # basic one with two linear layers and final output with sigmoid
    def __init__(self, input_size=8258, hidden_size=1024, num_layers=1):
        super().__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, dropout=0, num_layers=num_layers)
        self.fc1 = nn.Linear(hidden_size, 4000)
        self.fc2 = nn.Linear(4000, 4000)
        self.fc3 = nn.Linear(4000, input_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # hidden state
        h_0 = torch.zeros(self.num_layers * 1, x.size(0), self.hidden_size).to(x.device)  # Modified line
        # cell state
        c_0 = torch.zeros(self.num_layers * 1, x.size(0), self.hidden_size).to(x.device)  # Modified line
        # x = x.float()
        out, (hn, cn) = self.lstm(x, (h_0, c_0))
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        out = self.fc3(out)
        out = torch.sigmoid(out)

        return out

In [4]:
data_dir = "/sdf/group/lcls/ds/scratch/s2e_scratch/Data/SFG_intPhEn/version2"

In [5]:
model = LSTMModel1(num_layers=1, input_size=8267)

In [13]:
test_dataset = CustomSequenceTiming(data_dir, range(90,100), prefix="X_new")

In [14]:
test_dataset.shape

(1000, 10, 8267)

In [15]:
preds, time_elapsed = time_prediction(model, model_param_path="/sdf/group/lcls/ds/scratch/s2e_scratch/Data/SFG_RNN_0602/model1_intPhEn/model_epoch_29.pth", test_dataset=test_dataset, batch_size=200, use_gpu=True, data_parallel=False, verbose=1)

Predicting batch 1/5
iter 0
iter 10
iter 20
iter 30
iter 40
iter 50
iter 60
iter 70
iter 80
iter 90
Predicting batch 1/5
iter 0
iter 10
iter 20
iter 30
iter 40
iter 50
iter 60
iter 70
iter 80
iter 90
Predicting batch 1/5
iter 0
iter 10
iter 20
iter 30
iter 40
iter 50
iter 60
iter 70
iter 80
iter 90
Predicting batch 1/5
iter 0
iter 10
iter 20
iter 30
iter 40
iter 50
iter 60
iter 70
iter 80
iter 90
Predicting batch 1/5
iter 0
iter 10
iter 20
iter 30
iter 40
iter 50
iter 60
iter 70
iter 80
iter 90


In [16]:
time_elapsed

8.417204856872559